In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
courses = pd.read_csv('../data/Courses.csv', index_col=0)

In [3]:
with open('../data/study_programmes.txt') as f:
    study_programmes = f.read().split('_')

In [4]:
for program in study_programmes:
    courses[program.lower()] = [None for _ in range(len(courses))]

In [5]:
bachelor_course_types = {'naturvidenskabeligt grundfag', 'teknologisk linjefag', 'projekter og almene fag'}

kandidat_course_types = {'generel retningskompetence', 'teknologisk specialisering', }

diplom_course_types = {'tilvalgskurser', 'obligatorisk kursus', 'specialiseringskursus'}

In [6]:
for i, (kursus, number) in enumerate(zip(courses.Kursustype, courses['Kursus id'])):
    kursus = kursus.replace('_Kurset udbydes som enkeltfag', '').split('_')
    kursus_type = [kursus[0]]
    for line in kursus[1:]:
        if line.lower() == 'kandidat' or line.lower() == 'bachelor':
            kursus_type.append(line)
            
        else:
            program = None
            if line[:10].lower() == 'bachelor i':
                courses['kunstig intelligens og data (bac)'][i] = 'any' # FIX THIS LATER
            
            else:
                elements = line.split(', ')
                elements[0] = elements[0].replace('Naturvidenskabelige', 'Naturvidenskabeligt') # Spelling inconsistency
                elements[0] = elements[0].replace('Technology Core Courses', 'teknologisk linjefag') # Translation
                
                if len(elements) == 2:
                    if elements[0].lower() in diplom_course_types:
                        program = elements[1] + ' (dip)'
                    
                    elif elements[0].lower() in bachelor_course_types:
                        if elements[0].lower() == 'projekter og almene fag':
                            program = elements[1].replace('BSc i ', '') + ' (bac)'
                            
                        else:
                            program = elements[1] + ' (bac)'
                        
                        program = program.replace('&', 'og')

                    elif elements[0].lower() in kandidat_course_types:
                        program = elements[1] + ' (kan)'                            
                        
                
                elif len(elements) == 3:
                    if elements[1].lower() == 'kandidatuddannelsen':
                        program = (elements[2] + ' (kan)').replace('  ', ' ')                        
                            
                    elif elements[1].lower() == 'diplomingeniørretning':
                        program = elements[2] + ' (dip)'
                    
                    else:
                        program = elements[1]
                
                if program is None:
                    continue
                
                if program.lower() in courses.columns:
                    courses[program.lower()][i] = elements[0].lower()
                    
                else:
                    # Edge cases
                    if program[-9:] == 'Mobilitet':
                        courses['mobilitet, transport og logistik (dip)'][i] = elements[0].lower()
                    
                    elif program[-5:] == '(dip)':
                        courses['bygningsdesign (dip)'][i] = elements[0].lower()
                    
                    elif program[-5:] == '(kan)':
                        courses['bioinformatik og systembiologi (kan)'][i] = elements[0].lower()
                    
                    elif program[:6] == 'Desing':
                        courses['design og innovation (bac)'][i] = elements[0].lower()
                    
                    #else:
                        #print(f'{elements[0]:<30} --- {program.lower()}')
                        #print(number, program.lower())
            
    courses.Kursustype[i] = ', '.join(kursus_type)

In [7]:
# Fix location of courses
for i, placering in enumerate(courses['Undervisningens placering']):
    s = ' '.join(placering.split(' ')[:2])
    courses['Undervisningens placering'][i] = s if s == 'Campus Lyngby' or s == 'Campus Ballerup' else 'Unknown'

In [8]:
#for i, prereq in enumerate(courses['Anbefalede forudsætninger']):
#    if prereq is not np.nan and ('/14' in prereq or '/33' in prereq):
#        print(i, prereq, '\n')

In [9]:
# Fix recommended prerequisites
courses['Anbefalede forudsætninger'][190] = '_PROBLEM_Erfaring med Matlab-programmering(fx 02631/02633) og numeriskeberegninger (fx 02601).'
courses['Anbefalede forudsætninger'][1506] = 'Det forudsættes at kurset 62531 og 02312/02314 følges samtidig,eller at man har tilsvarendeforudsætninger'
courses['Anbefalede forudsætninger'][417] = ''

for i, (forudsætning, number) in enumerate(zip(courses['Anbefalede forudsætninger'], courses['Kursus id'])):
    if forudsætning is not np.nan:
        x = re.findall(r'(\d{5})', forudsætning)
        y = re.findall(r'(\d{2}[A-Z]\d{2})', forudsætning)
        fs = set(kursus for kursus in x + y if kursus != number)
        courses['Anbefalede forudsætninger'][i] = ','.join(fs) if len(fs) else 'Unknown'


In [10]:
# Fix department
for i, institut in enumerate(courses.Institut):
    change = False
    
    # Fix spelling errors
    if not institut[2] == ' ':
        institut = institut[:2] + ' ' + institut[2:]
        change = True
    
    if institut[3] == 'I':
        if institut[11] != ' ':
            institut = institut[:11] + ' ' + institut[11:]
            change = True
        elif institut[15] != ' ':
            institut = institut[:15] + ' ' + institut[15:]
            change = True
    
    if change:
        courses.Institut[i] = institut

In [11]:
for i, points in enumerate(courses['Point( ECTS )']):
    if ',' in points:
        courses['Point( ECTS )'][i] = float(points.replace(',', '.'))
    else:
        courses['Point( ECTS )'][i] = int(points)

In [12]:
courses.to_csv('../data/UpdatedCourses.csv')